In [1]:
import tensorflow as tf
import libspn as spn
sess = tf.InteractiveSession()

# Product in log space

In [2]:
v1 = 0
v2 = 0
v1_log=tf.log(tf.constant(v1, dtype=tf.float32))
v2_log=tf.log(tf.constant(v2, dtype=tf.float32))
print(v1_log.eval())
print(v2_log.eval())

-inf
-inf


In [3]:
out_log = tf.add(v1_log, v2_log)
print(out_log.eval())

-inf


In [4]:
out = tf.exp(out_log)
print(out.eval())
print(v1*v2)

0.0
0


# Sum in log space (naive)

In [5]:
v1 = 0
v2 = 0
v1_log=tf.log(tf.constant(v1, dtype=tf.float32))
v2_log=tf.log(tf.constant(v2, dtype=tf.float32))
print(v1_log.eval())
print(v2_log.eval())

-inf
-inf


In [6]:
v_log_max = tf.maximum(v1_log, v2_log)
print(v_log_max.eval())

-inf


In [7]:
v1_log_rebased = tf.sub(v1_log, v_log_max)
v2_log_rebased = tf.sub(v2_log, v_log_max)
print(v1_log_rebased.eval())
print(v2_log_rebased.eval())

nan
nan


In [8]:
out_log = v_log_max + tf.log(tf.add(tf.exp(v1_log_rebased), tf.exp(v2_log_rebased)))
print(out_log.eval())

nan


In [9]:
out = tf.exp(out_log)
print(out.eval())
print(v1+v2)

nan
0


This will produce nan if all operants are 0. Then the v_log_max ends up being neg inf and subtracting neg inf from neg info results in nan. This case must be detected.

# Sum in log space (with all zero detection)

In [10]:
v1 = 0
v2 = 0
v1_log=tf.log(tf.constant(v1, dtype=tf.float32))
v2_log=tf.log(tf.constant(v2, dtype=tf.float32))
print(v1_log.eval())
print(v2_log.eval())

-inf
-inf


In [11]:
v_log_max = tf.maximum(v1_log, v2_log)
print(v_log_max.eval())

-inf


In [12]:
import math
all_zero = tf.equal(v_log_max, tf.constant(-math.inf, dtype=tf.float32))
print(all_zero.eval())

True


In [13]:
v1_log_rebased = tf.sub(v1_log, v_log_max)
v2_log_rebased = tf.sub(v2_log, v_log_max)
print(v1_log_rebased.eval())
print(v2_log_rebased.eval())

nan
nan


In [14]:
out_log_normal = v_log_max + tf.log(tf.add(tf.exp(v1_log_rebased), tf.exp(v2_log_rebased)))
out_log_zeros = tf.constant(-math.inf, dtype=tf.float32)
print(out_log_normal.eval())
print(out_log_zeros.eval())
out_log = tf.select(all_zero, out_log_zeros, out_log_normal)
print(out_log.eval())

nan
-inf
-inf


In [15]:
out = tf.exp(out_log)
print(out.eval())
print(v1+v2)

0.0
0


# Sum in log space on a batch

In [16]:
v11 = 0
v12 = 0
v21 = 0.5
v22 = 0.6
v_batch = tf.constant([[v11,v12], [v21, v22]], dtype=tf.float32)
v_log=tf.log(v_batch)
print(v_log.eval())

[[       -inf        -inf]
 [-0.69314718 -0.51082557]]


In [17]:
v_log_max = tf.reduce_max(v_log, 1, keep_dims=True)
print(v_log_max.eval())

[[ -3.40282347e+38]
 [ -5.10825574e-01]]


In [18]:
import math
all_zero = tf.equal(v_log_max, tf.constant(-math.inf, dtype=tf.float32))
print(all_zero.eval())

[[False]
 [False]]


In [19]:
v_log_rebased = tf.sub(v_log, v_log_max)
print(v_log_rebased.eval())

[[       -inf        -inf]
 [-0.18232161  0.        ]]


In [20]:
out_log_normal = v_log_max + tf.log(tf.reduce_sum(tf.exp(v_log_rebased), 1, keep_dims=True))
print(tf.shape(out_log_normal).eval())
out_log_zeros = tf.fill(tf.shape(out_log_normal), tf.constant(-math.inf, dtype=tf.float32))
print(out_log_normal.eval())
print(out_log_zeros.eval())
out_log = tf.select(all_zero, out_log_zeros, out_log_normal)
print(out_log.eval())

[2 1]
[[       -inf]
 [ 0.09531021]]
[[-inf]
 [-inf]]
[[       -inf]
 [ 0.09531021]]


In [21]:
out = tf.exp(out_log_normal)
out_arr = out.eval()
print(out_arr)
print(tf.reduce_sum(v_batch, 1, keep_dims=True).eval())

[[ 0.        ]
 [ 1.10000002]]
[[ 0.        ]
 [ 1.10000002]]


In [22]:
out_arr == 0.0

array([[ True],
       [False]], dtype=bool)

In [23]:
print(tf.maximum(-math.inf, -math.inf).eval())
print(tf.reduce_max([-math.inf, -math.inf]).eval())
np.finfo(np.float32)

-inf
-3.40282e+38


finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

There clearly is a problem with reduce_max which returns min float32 instead of -inf for both negative infinity inputs. At the same time, tf.maximum works as expected. This still gives the correct result, and actually will lead to a simpler code since the negative inf detection is not needed in this case. But is this a stable behavior or a bug that will be removed?

# Sum in log space on a batch with weights

In [24]:
v11 = 0
v12 = 0
v21 = 0.0000000001
v22 = 0.0000000001
w1 = 0.1
w2 = 0.9
v_batch = tf.constant([[v11,v12], [v21, v22]], dtype=tf.float32)
w = tf.constant([w1, w2], dtype=tf.float32)
v_log = tf.log(v_batch)
w_log = tf.log(w)
print(v_log.eval())
print(w_log.eval())

[[       -inf        -inf]
 [-23.0258503 -23.0258503]]
[-2.30258512 -0.10536055]


In [25]:
v_log = tf.add(v_log, w_log)

In [26]:
v_log_max = tf.reduce_max(v_log, 1, keep_dims=True)
print(v_log_max.eval())

[[ -3.40282347e+38]
 [ -2.31312103e+01]]


In [27]:
import math
all_zero = tf.equal(v_log_max, tf.constant(-math.inf, dtype=tf.float32))
print(all_zero.eval())

[[False]
 [False]]


In [28]:
v_log_rebased = tf.sub(v_log, v_log_max)
print(v_log_rebased.eval())

[[       -inf        -inf]
 [-2.19722557  0.        ]]


In [29]:
out_log_normal = v_log_max + tf.log(tf.reduce_sum(tf.exp(v_log_rebased), 1, keep_dims=True))
print(tf.shape(out_log_normal).eval())
out_log_zeros = tf.fill(tf.shape(out_log_normal), tf.constant(-math.inf, dtype=tf.float32))
print(out_log_normal.eval())
print(out_log_zeros.eval())
out_log = tf.select(all_zero, out_log_zeros, out_log_normal)
print(out_log.eval())

[2 1]
[[       -inf]
 [-23.0258503]]
[[-inf]
 [-inf]]
[[       -inf]
 [-23.0258503]]


In [30]:
out = tf.exp(out_log_normal)
out_arr = out.eval()
print(out_arr)
print([[np.float32(v11)*np.float32(w1)+np.float32(v12)*np.float32(w2)], 
       [np.float32(v21)*np.float32(w1)+np.float32(v22)*np.float32(w2)]])
print([[np.exp(np.log(np.float32(v11))+np.log(np.float32(w1)))+np.exp(np.log(np.float32(v12))+np.log(np.float32(w2)))], 
       [np.exp(np.log(np.float32(v21))+np.log(np.float32(w1)))+np.exp(np.log(np.float32(v22))+np.log(np.float32(w2)))]])

[[  0.00000000e+00]
 [  1.00000064e-10]]
[[0.0], [1e-10]]
[[0.0], [1.000001e-10]]


/home/czikus/.local/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in log
